In [1]:
import numpy as np 
import pandas as pd 

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("Real estate.csv")
df.rename({'Ouse_price_of_unit_area': 'House_price_of_unit_area'}, axis = 1, inplace = True)
df.drop("No", axis = 1, inplace = True)
column_maping = {}
for i in df.columns:
    new_column = i[3:].capitalize().replace(' ', '_')
    column_maping[i] = new_column
# Now we will rename the column using the dictinary 
df.rename(columns = column_maping, inplace = True)
df.rename({'Ouse_price_of_unit_area': 'House_price_of_unit_area'}, axis = 1, inplace = True)
X  = df.drop(['Transaction_date', "House_price_of_unit_area"], axis = 1)
y = df['House_price_of_unit_area']

In [3]:

# Do not change the code below
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
X_scale = np.asarray(X_scale)
y = np.asarray(y)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.2, random_state = 42)
# Do not change the code Above

# Add a column of ones to X_train and X_test to account for the bias term (intercept)
X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# Perform SVD decomposition on the training data
U, s, VT = np.linalg.svd(X_train_b, full_matrices=False)

# Create diagonal matrix for Sigma
S_diag = np.diag(s)

# Compute the pseudo-inverse of the training data
X_train_pinv = VT.T @ np.linalg.inv(S_diag) @ U.T

# Calculate the weights (regression coefficients), including the bias term (intercept)
w = X_train_pinv @ y_train

# Make predictions using the testing set
y_pred = X_test_b @ w

se = (y_pred-y_test) ** 2
mse = se.mean()
rmse = mse**0.5
print(f"Root Mean Squre Error using SVD {rmse}")

Root Mean Squre Error using SVD 7.38789179677546


In [4]:
class Model(object):
    """
     Ridge Regression.
    """

    def fit(self, X, y, alpha=0):
        """
        Fits the ridge regression model to the training data.

        Arguments
        ----------
        X: nxp matrix of n examples with p independent variables
        y: response variable vector for n examples
        alpha: regularization parameter.
        """
      
        intercept = np.ones((len(X),1))
        X_b = np.c_[intercept,X]
        
        I = np.identity(X_b.shape[1])
        
        betha_optim = np.linalg.inv(X_b.T@X_b + alpha*I) @ np.dot(X_b.T,y)
        self.betas = betha_optim
        return betha_optim
   
    def predict(self, X):
        """
        Predicts the dependent variable of new data using the model.

        Arguments
        ----------
        X: nxp matrix of n examples with p covariates

        Returns
        ----------
        response variable vector for n examples
        """
       	# Your code here
        X_predictor = np.c_[np.ones((X.shape[0], 1)), X]
        self.predictions = X_predictor @ self.betas
        return self.predictions

    def rmse(self, X, y):
        """
        Returns the RMSE(Root Mean Squared Error) when the model is validated.
            
        Arguments
        ----------
        X: nxp matrix of n examples with p covariates
        y: response variable vector for n examples
            
        Returns
        ----------
        RMSE when model is used to predict y
        """
        y_predict = self.predict(X=X)
        se = (y_predict-y) ** 2
        mse = se.mean()
        rmse = mse**0.5
        return rmse

# Do not change the code below
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
X_scale = np.asarray(X_scale)
y = np.asarray(y)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.2, random_state = 42)
my_model = Model()
my_model.fit(X=X_train, y=y_train, alpha=0.01)
rmse = my_model.rmse(X=X_test, y=y_test)
print(f"Root Mean Squre Error Ridge {rmse}")

Root Mean Squre Error Ridge 7.387798950430713


In [7]:
import numpy as np

# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_derivative(x):
    return sigmoid(x) * (1-sigmoid(x))

# Mean Squared Error loss
def mse_loss(y_true, y_pred): 
    mse_err = np.sum(np.square(y_true - y_pred))/len(y_true)
    return mse_err

# Forward pass
def forward_pass(x, W_h1, b_h1, W_o, b_o):
    # The output of the hidden layer
    a_h1 = np.dot(x, W_h1) + b_h1
    # Apply activation to the output of the hidden layer
    z_h1 = sigmoid(a_h1)
    # Use ouput of activation as input to the output layer (it is just similar to first layer but we don't apply activation)
    y_pred = np.dot(z_h1, W_o) + b_o
    return y_pred, z_h1, a_h1

# Backward pass
def backward_pass(x, y_true, y_pred, z_h1, a_h1, W_h1, W_o):
    
    # Derivative of loss with respect to y_pred
    dL_dy_pred = 2 * (y_pred - y_true) / len(y_true)
    
    # Gradients for output layer
    dL_dW_o = np.dot(z_h1.T, dL_dy_pred)
    dL_db_o = np.sum(dL_dy_pred, axis = 0, keepdims=True)
    
    # Derivative of loss with respect to z_h1
    dL_dz_h1 = np.dot(dL_dy_pred, W_o.T)
    #TODO derivative with respect to the output of activation layer
    
    # Derivative of loss with respect to a_h1
    dL_da_h1 = dL_dz_h1 * sigmoid_derivative(a_h1) # Derivative with respect to the activation layer
    
    # Gradients for hidden layer
    dL_dW_h1 = np.dot(x.T, dL_da_h1)
    dL_db_h1 = np.sum(dL_da_h1, axis=0,keepdims=True) 
    
    return dL_dW_h1, dL_db_h1, dL_dW_o, dL_db_o



scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
X_scale = np.asarray(X_scale)
y = np.asarray(y)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.2, random_state = 42)


y_train = y_train.reshape((y_train.shape[0],1))


# Network architecture
input_size = 5 # Number of features
hidden_layer_size = 20 # Number of neurons in layer
output_size = 1 # predicted variable

# Initial random weights and biases for each layer
W_h1 = np.random.randn(input_size, hidden_layer_size) * 0.001
b_h1 = np.zeros((1, hidden_layer_size))
W_o = np.random.randn(hidden_layer_size, output_size) * 0.001
b_o = np.zeros((1, output_size))

learning_rate = 0.2

#To save the weights which give the lowest loss
lowest_loss = float('inf')
best_weights = None


for i in range(100):
    # Forward pass to get predictions
    y_pred, z_h1, a_h1 = forward_pass(X_train, W_h1, b_h1, W_o, b_o)
    loss = mse_loss(y_train, y_pred)

    if loss < lowest_loss:
        lowest_loss = loss
        # Save the best weights and biases
        best_weights = (W_h1.copy(), b_h1.copy(), W_o.copy(), b_o.copy())

    # Backward pass to get gradients
    dL_dW_h1, dL_db_h1, dL_dW_o, dL_db_o = backward_pass(X_train, y_train, y_pred, z_h1, a_h1, W_h1, W_o)

    # Now you would use the gradients to update the weights and biases
    W_h1 -= learning_rate * dL_dW_h1
    b_h1 -= learning_rate * dL_db_h1
    W_o -= learning_rate * dL_dW_o
    b_o -= learning_rate * dL_db_o


W_h1_best, b_h1_best, W_o_best, b_o_best = best_weights
y_pred_test, _, _ = forward_pass(X_test, W_h1_best, b_h1_best, W_o_best, b_o_best)
se = (y_pred_test-y_test) ** 2
mse = se.mean()
rmse = mse**0.5
print(f"Root Mean Squre Error 1 Layer MLP {rmse}")

Root Mean Squre Error 1 Layer MLP 13.11371619563616


#### Bonus Lets add one more hidden layer (10 Points) 
### You must write down all gradients and complete the code below to get full bonus points


In [ ]:


# Forward pass
def forward_pass(x, W_h1, b_h1, W_h2, b_h2, W_o, b_o):
    a_h1 = #TODO compute ouput of first hidden layer
    z_h1 = sigmoid(a_h1) # apply activation to ouputs of first hidden layer
    
    a_h2 = #TODO compute ouput of second hidden layer
    z_h2 = sigmoid(a_h2) #apply activation to ouputs of first hidden layer
    
    y_pred = # compute ouput of output layer, why don't we apply activation ?
    
    return y_pred, z_h1, a_h1, z_h2, a_h2

# Backward pass
def backward_pass(x, y_true, y_pred, z_h1, a_h1, z_h2, a_h2, W_h1, W_h2, W_o):
    
    # Derivative of loss with respect to y_pred, what kind of loss are we using ?
    dL_dy_pred = #TODO
    
    # Gradients for output layer
    dL_dW_o = #TODO
    dL_db_o = #TODO
    
    # Derivative of loss with respect to z_h2
    dL_dz_h2 =  #TODO
    
    # Derivative of loss with respect to a_h2
    dL_da_h2 =  #TODO
    
    # Gradients for second hidden layer
    dL_dW_h2 =  #TODO
    dL_db_h2 =  #TODO
    
    # Derivative of loss with respect to z_h1
    dL_dz_h1 =  #TODO
    
    # Derivative of loss with respect to a_h1
    dL_da_h1 =  #TODO
    
    # Gradients for first hidden layer
    dL_dW_h1 =  #TODO
    dL_db_h1 =  #TODO
    
    return dL_dW_h1, dL_db_h1, dL_dW_h2, dL_db_h2, dL_dW_o, dL_db_o



# Random input and true output (modify these according to your dataset)
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
X_scale = np.asarray(X_scale)
y = np.asarray(y)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.2, random_state = 42)
y_train = y_train.reshape((y_train.shape[0],1))

# Network architecture
input_size = 5 # Number of features
hidden_layer1_size = 100
hidden_layer2_size = 20
output_size = 1

# Initial random weights and biases for each layer
W_h1 = np.random.randn(input_size, hidden_layer1_size) * 0.001
b_h1 = np.zeros((1, hidden_layer1_size))
W_h2 = np.random.randn(hidden_layer1_size, hidden_layer2_size) * 0.001
b_h2 = np.zeros((1, hidden_layer2_size))
W_o = np.random.randn(hidden_layer2_size, output_size) * 0.001
b_o = np.zeros((1, output_size))


learning_rate = 0.1

#Training loop
for i in range(200):
    # Forward pass to get predictions
    y_pred, z_h1, a_h1, z_h2, a_h2 = forward_pass(X_train, W_h1, b_h1, W_h2, b_h2, W_o, b_o)

    #TODO Compute the loss
    # Backward pass to get gradients
    dL_dW_h1, dL_db_h1, dL_dW_h2, dL_db_h2, dL_dW_o, dL_db_o = backward_pass(X_train, y_train, y_pred, z_h1, a_h1, z_h2, a_h2, W_h1, W_h2, W_o)

    # Now you would use the gradients to update the weights and biases for each layer
    
    W_h1 -= #TODO
    b_h1 -= #TODO
    W_h2 -= #TODO
    b_h2 -= #TODO
    W_o -= #TODO
    b_o -= #TODO

y_pred, _, _, _, _ = forward_pass(X_test, W_h1, b_h1, W_h2, b_h2, W_o, b_o)
se = (y_pred-y_test) ** 2
mse = se.mean()
rmse = mse**0.5
print(f"Root Mean Squre Error 2 Layer MLP {rmse}")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping


model = Sequential()


model.add(Dense(400, input_dim = 5, kernel_initializer = 'he_uniform',  activation = 'relu')) #
model.add(Dropout(0.2))


model.add(Dense(400, input_dim = 5, kernel_initializer = 'he_uniform', activation = 'relu')) #
model.add(Dropout(0.2))

model.add(Dense(400, kernel_initializer = 'he_uniform',activation = 'relu')) #
model.add(Dropout(0.2))

model.add(Dense(1, activation = 'linear'))

model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.summary()

In [ ]:
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)
X_scale = np.asarray(X_scale)
y = np.asarray(y)
X_train, X_test, y_train, y_test = train_test_split(X_scale, y, test_size = 0.2, random_state = 42)
y_train = y_train.reshape((y_train.shape[0],1))
history = model.fit(X_train, y_train,  epochs = 200, validation_data=(X_test, y_test), 
                    callbacks = EarlyStopping(monitor = 'val_loss',patience = 40))

In [ ]:
print(f"RMSE error using deeper neural network {model.evaluate(X_test, y_test)**0.5}")